In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import json
from datetime import datetime

class AlmacenGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema de Gestión de Almacenes")
        self.root.geometry("800x600")
        
        # Cargar datos iniciales
        self.cargar_datos()
        
        # Crear pestañas
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        
        # Pestaña de Visualización
        self.tab_visualizar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_visualizar, text='Visualizar Inventario')
        
        # Pestaña de Inserción
        self.tab_insertar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_insertar, text='Insertar Producto')
        
        # Pestaña de Búsqueda
        self.tab_buscar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_buscar, text='Buscar Producto')
        
        self.setup_visualizar_tab()
        self.setup_insertar_tab()
        self.setup_buscar_tab()
        
    def cargar_datos(self):
        try:
            with open('inventario.txt', 'r') as file:
                contenido = file.read()
                # Convertir el contenido en una lista de diccionarios
                self.inventario = []
                for item in contenido.strip().split('}'):
                    if item.strip():
                        # Asegurarse de que el item termine con }
                        if not item.endswith('}'):
                            item += '}'
                        # Limpiar el item de espacios y caracteres no deseados
                        item = item.strip().lstrip('{').strip()
                        if item:
                            try:
                                self.inventario.append(json.loads('{' + item))
                            except json.JSONDecodeError:
                                continue
        except FileNotFoundError:
            self.inventario = []
            messagebox.showwarning("Advertencia", "No se encontró el archivo inventarios1.txt")
    
    def guardar_datos(self):
        with open('inventario.txt', 'w') as file:
            for item in self.inventario:
                file.write(str(item))
    
    def setup_visualizar_tab(self):
        # Crear Treeview
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree = ttk.Treeview(self.tab_visualizar, columns=columns, show='headings')

        # Configurar columnas
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        # Agregar scrollbar
        scrollbar = ttk.Scrollbar(self.tab_visualizar, orient='vertical', command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)

        # Posicionar elementos
        self.tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')

        # Crear botón para eliminar
        ttk.Button(self.tab_visualizar, text="Eliminar Producto", command=self.eliminar_item).pack(side='bottom', pady=10)

        # Cargar datos en el Treeview
        self.actualizar_treeview()

    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar datos
        for producto in self.inventario:
            self.tree.insert('', 'end', values=(
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
    
    def setup_insertar_tab(self):
        # Crear formulario
        ttk.Label(self.tab_insertar, text="ID:").grid(row=0, column=0, padx=5, pady=5)
        self.id_entry = ttk.Entry(self.tab_insertar)
        self.id_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Nombre:").grid(row=1, column=0, padx=5, pady=5)
        self.nombre_entry = ttk.Entry(self.tab_insertar)
        self.nombre_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Tipo:").grid(row=2, column=0, padx=5, pady=5)
        self.tipo_combo = ttk.Combobox(self.tab_insertar, values=['perecedero', 'no perecedero'])
        self.tipo_combo.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Cantidad:").grid(row=3, column=0, padx=5, pady=5)
        self.cantidad_entry = ttk.Entry(self.tab_insertar)
        self.cantidad_entry.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Fecha Vencimiento:").grid(row=4, column=0, padx=5, pady=5)
        self.fecha_entry = ttk.Entry(self.tab_insertar)
        self.fecha_entry.grid(row=4, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_insertar, text="Insertar Producto", command=self.insertar_producto).grid(row=5, column=0, columnspan=2, pady=20)
    
    def insertar_producto(self):
        try:
            nuevo_producto = {
                'id': int(self.id_entry.get()),
                'nombre': self.nombre_entry.get(),
                'tipo': self.tipo_combo.get(),
                'cantidad': int(self.cantidad_entry.get()),
                'fech_venc': self.fecha_entry.get()
            }
            
            # Validaciones básicas
            if not nuevo_producto['nombre'] or not nuevo_producto['tipo']:
                messagebox.showerror("Error", "Todos los campos son obligatorios")
                return
            
            # Verificar si el ID ya existe
            if any(p['id'] == nuevo_producto['id'] for p in self.inventario):
                messagebox.showerror("Error", "El ID ya existe")
                return
            
            self.inventario.append(nuevo_producto)
            self.guardar_datos()
            self.actualizar_treeview()
            
            # Limpiar campos
            self.id_entry.delete(0, 'end')
            self.nombre_entry.delete(0, 'end')
            self.tipo_combo.set('')
            self.cantidad_entry.delete(0, 'end')
            self.fecha_entry.delete(0, 'end')
            
            messagebox.showinfo("Éxito", "Producto insertado correctamente")
            
        except ValueError:
            messagebox.showerror("Error", "Por favor, ingrese valores válidos")
    
    def setup_buscar_tab(self):
        # Crear campos de búsqueda
        ttk.Label(self.tab_buscar, text="Buscar por nombre:").grid(row=0, column=0, padx=5, pady=5)
        self.buscar_nombre_entry = ttk.Entry(self.tab_buscar)
        self.buscar_nombre_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_buscar, text="Buscar por tipo:").grid(row=1, column=0, padx=5, pady=5)
        self.buscar_tipo_combo = ttk.Combobox(self.tab_buscar, values=['perecedero', 'no perecedero'])
        self.buscar_tipo_combo.grid(row=1, column=1, padx=5, pady=5)
        ttk.Button(self.tab_buscar, text="Buscar", command=self.buscar_productos).grid(row=2, column=0, columnspan=2, pady=20)

        # Crear Treeview para resultados
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree_busqueda = ttk.Treeview(self.tab_buscar, columns=columns, show='headings')
        
        for col in columns:
            self.tree_busqueda.heading(col, text=col)
            self.tree_busqueda.column(col, width=100)
        
        self.tree_busqueda.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        
        # Crear botón de eliminación de búsqueda
        ttk.Button(self.tab_buscar, text="Eliminar Producto", command=self.eliminar_item_busqueda).grid(row=4, column=0, columnspan=2, pady=10) 
    
    def eliminar_item(self):
        # Obtener el item seleccionado
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Advertencia", "Debe seleccionar un producto para eliminar.")
            return
        
        # Eliminar item
        item_values = self.tree.item(selected_item, 'values')
        item_id = int(item_values[0])
        
        self.inventario = [p for p in self.inventario if p['id'] != item_id]
        self.guardar_datos()
        self.actualizar_treeview()
        messagebox.showinfo("Éxito", "Producto eliminado correctamente")
    
    def eliminar_item_busqueda(self):
        # Obtener el item seleccionado
        selected_item = self.tree_busqueda.selection()
        if not selected_item:
            messagebox.showwarning("Advertencia", "Debe seleccionar un producto para eliminar.")
            return
        
        item_values = self.tree_busqueda.item(selected_item, 'values')
        item_id = int(item_values[0])
        
        self.inventario = [p for p in self.inventario if p['id'] != item_id]
        self.guardar_datos()
        self.actualizar_treeview()
        self.buscar_productos()  # Actualizar la búsqueda con los datos restantes
        messagebox.showinfo("Éxito", "Producto eliminado correctamente")
    
    def buscar_productos(self):
    # Limpiar resultados anteriores
        for item in self.tree_busqueda.get_children():
            self.tree_busqueda.delete(item)
    
    # Obtener filtros
        nombre = self.buscar_nombre_entry.get().lower()
        tipo = self.buscar_tipo_combo.get().lower()
    
    # Filtrar productos
        productos_filtrados = [
            p for p in self.inventario
            if (not nombre or nombre in p['nombre'].lower()) and (not tipo or tipo == p['tipo'].lower())
        ]
    
    # Insertar resultados en el Treeview
        for producto in productos_filtrados:
            self.tree_busqueda.insert('', 'end', values=(
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
        ))


if __name__ == "__main__":
    root = tk.Tk()
    app = AlmacenGUI(root)
    root.mainloop()  